In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [167]:
# sub = pd.read_csv('./data/sample_submission.csv')
# sub.head()

In [168]:
# test_df = pd.read_csv('./data/test.csv')
# test_df.head()

In [169]:
df = pd.read_csv('./train.csv')
df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [170]:
df.shape

(750000, 9)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB


In [172]:
df.describe()

,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,216506.495284,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,187499.750000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,374999.500000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,562499.250000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,749999.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


In [173]:
df['Sex'] = df['Sex'].map({'male':0, 'female':1})

In [174]:
df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,0,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,1,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,1,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,0,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,1,38,166.0,61.0,25.0,102.0,40.6,146.0


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  int64  
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 51.5 MB


In [176]:
X = df.drop(['Calories','id'], axis=1)
y = df['Calories']

In [177]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9684446417027912

In [178]:
from sklearn.metrics import mean_squared_log_error
y_pred = model.predict(X_test)
mean_squared_log_error(np.log1p(y_test), y_pred)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain values less than or equal to -1.

In [179]:
# due to this above error we will take log(1+y) during training and avoid negatives
y_log = np.log1p(y_train)
model.fit(X_train, y_log)
model.score(X_test, np.log1p(y_test))

0.9653113210923763

In [180]:
import xgboost as xgb
xgb = xgb.XGBRegressor()
xgb.fit(X_train, y_log)
xgb.score(X_test, np.log1p(y_test))

0.9957682649383526

In [181]:
from sklearn.model_selection import KFold,cross_val_score

cv = KFold(n_splits=5)
kfold = cross_val_score(LinearRegression(), X, np.log1p(y), cv=cv)
print(kfold)
print(kfold.mean())

[0.9653188  0.96505141 0.96485866 0.96489158 0.9654416 ]
0.9651124115472214


In [182]:
y_pred = model.predict(X_test)
mean_squared_log_error(np.log1p(y_test), y_pred)

0.0020361134222748834

In [183]:
y_pred = xgb.predict(X_test)
mean_squared_log_error(np.log1p(y_test), y_pred)

0.0003182338803635834

In [184]:
xgb.feature_importances_

array([1.2877727e-02, 1.0597401e-02, 2.1635808e-04, 1.8418327e-03,
       9.4550580e-01, 2.0244695e-02, 8.7161809e-03], dtype=float32)

In [185]:
fe = pd.DataFrame({'feature':X.columns, 'importance':xgb.feature_importances_})
fe.sort_values(by='importance', ascending=False)
fe.head(10)

,feature,importance
0,Sex,0.012878
1,Age,0.010597
2,Height,0.000216
3,Weight,0.001842
4,Duration,0.945506
5,Heart_Rate,0.020245
6,Body_Temp,0.008716


In [186]:
test_data = pd.read_csv('./test.csv')
test_data.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [187]:
test_data['Sex'] = test_data['Sex'].map({'male':0, 'female':1})

In [188]:
X.describe()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,0.500961,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253
std,0.499999,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000
25%,0.000000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000
50%,1.000000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000
75%,1.000000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000
max,1.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000


In [189]:
test_data.describe()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,874999.500000,0.501124,41.452464,174.725624,75.147712,15.415428,95.479084,40.036093
std,72168.927986,0.500000,15.177769,12.822039,13.979513,8.349133,9.450161,0.778448
min,750000.000000,0.000000,20.000000,127.000000,39.000000,1.000000,67.000000,37.100000
25%,812499.750000,0.000000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000
50%,874999.500000,1.000000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000
75%,937499.250000,1.000000,52.000000,185.000000,87.000000,23.000000,103.000000,40.600000
max,999999.000000,1.000000,79.000000,219.000000,126.000000,30.000000,128.000000,41.500000


In [190]:
id = test_data['id']
test_data = test_data.drop(['id'], axis=1)

In [195]:
submision = model.predict(test_data)

In [196]:
sub_df = pd.DataFrame({'id':id, 'Calories':submision})
sub_df.head()

,id,Calories
0,750000,3.544088
1,750001,4.601120
2,750002,4.525110
3,750003,4.946319
4,750004,4.382525


In [197]:
# prompt: [sub_df['Calories'] < 0].count()
# i want count

print(sum(sub_df['Calories'] < 0))

0


In [198]:
pd.DataFrame({'id':id, 'Calories':submision}).to_csv('submission.csv', index=False)